In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader, prep_config
from pure_funcs import denumpyize, numpyize, get_template_live_config, candidate_to_live_config, calc_spans, get_template_live_config, analyze_fills, \
    create_xk, round_dynamic
from njit_funcs import calc_emas, calc_long_orders, round_, calc_bankruptcy_price, calc_diff
from procedures import dump_live_config, load_live_config, add_argparse_args, prep_config, make_get_filepath
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = 'configs/backtest/default.hjson'
        self.optimize_config_path = 'configs/optimize/default.hjson'
        self.exchange = 'binance'
        self.symbol = 'XMRETH'
        self.market_type = 'spot'
        self.user = 'your_user_name'
        self.start_date = '2021-01-01'
        self.end_date = '2021-07-01'
        self.starting_balance = 100.0
config = await prep_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_sampled_ticks()
timestamps = data[:, 0]
qtys = data[:, 1]
prices = data[:, 2]
config['n_days'] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f'millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms')

In [ ]:
df = pd.DataFrame({'timestamp': timestamps, 'qty': qtys, 'price': prices}).set_index('timestamp')
df

In [ ]:
# choose a slice on which to test
wsize_days = 240
ts = int(data[-1][0] - 60 * 60 * 24 * 1000 * wsize_days)
idx = np.argmax(data[:, 0] >= ts)
dataslice = data[idx:]

In [ ]:
template = \
    {"config_name": "hand_tuned",
     "logging_level": 0,
     "long": {"enabled": True,
              "iprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "iprc_const": 0.995,
              "iqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "iqty_const": 0.005,
              "pbr_limit": 1.8,
              "markup_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "markup_const": 1.005,
              "rprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "rprc_PBr_coeffs": [[-0.0, -0.2]],
              "rprc_const": 0.995,
              "rqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "rqty_const": 1.3,
              "pbr_stop_loss": 0.2},
     "max_span": 60 * 24 * 2,
     "min_span": 60 * 7,
     "n_spans": 3,
     "shrt": {"enabled": True,
              "iprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "iprc_const": 1.01,
              "iqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "iqty_const": 0.005,
              "pbr_limit": 1.8,
              "markup_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "markup_const": 0.995,
              "rprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "rprc_PBr_coeffs": [[0.0, 0.2]],
              "rprc_const": 1.005,
              "rqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "rqty_const": 1.3,
              "pbr_stop_loss": 0.2}}

config['starting_balance'] = 1000.0

config_to_test = {**config, **numpyize(template)}
dump_live_config(config_to_test, make_get_filepath('tmp/hand_tuned.json'))

In [ ]:
sts = time()
fills, info = backtest(config_to_test, dataslice)
elapsed = time() - sts
print(f'seconds elapsed {elapsed:.4f}')
print(f'did finish {info[0]}, lowest eqbal ratio {info[1]:.4f}, closest bkr {info[2]:.4f}')
result = {**config_to_test, **{'lowest_eqbal_ratio': info[1], 'closest_bkr': info[2]}}
sample_size_ms = dataslice[1][0] - dataslice[0][0]
max_span_ito_n_samples = int(config_to_test['max_span'] * 60 / (sample_size_ms / 1000))
fdf, analysis = analyze_fills(fills, result, dataslice[max_span_ito_n_samples][0], dataslice[-1][0])
pprint.pprint(analysis)
fdf

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()

In [ ]:
fdf.psize.plot()

In [ ]:
periodic_gain_n_days = 1
groups = fdf.groupby(fdf.timestamp // (1000 * 60 * 60 * 24 * periodic_gain_n_days))
pgs = groups.pnl.sum() / groups.balance.first()
pgs = pgs.reindex(np.arange(pgs.index[0], pgs.index[-1])).fillna(0.0)
pg_mean = pgs.mean()
pgs.plot()
plt.plot([pgs.index[0], pgs.index[-1]], [pg_mean, pg_mean])
print(f'mean {pg_mean:.4f}')
pgs

In [ ]:

# plot MA bands given spans
spans = calc_spans(*[config_to_test[k] for k in ['min_span', 'max_span', 'n_spans']])
print('spans', spans)
emas = pd.DataFrame({str(span): df.price.ewm(span=span * 60, adjust=False).mean() for span in spans})
lband = emas.min(axis=1)
uband = emas.max(axis=1)
df.price.iloc[::100].plot()
uband.iloc[::100].plot()
lband.iloc[::100].plot()


In [ ]:
# manual grid design
# modify parameters to see resulting grid
# long only

In [ ]:
xk = create_xk(config_to_test)
spot = xk['spot']
inverse = xk['inverse']
qty_step = xk['qty_step']
price_step = xk['price_step']
min_qty = xk['min_qty']
min_cost = xk['min_cost']
c_mult = xk['c_mult']
pbr_stop_loss = xk['pbr_stop_loss'][0]
pbr_limit = xk['pbr_limit'][0]
iqty_const = xk['iqty_const'][0]
iprc_const = xk['iprc_const'][0]
rqty_const = xk['rqty_const'][0]
rprc_const = xk['rprc_const'][0]
markup_const = xk['markup_const'][0]
iqty_MAr_coeffs = xk['iqty_MAr_coeffs'][0]
iprc_MAr_coeffs = xk['iprc_MAr_coeffs'][0]
rprc_PBr_coeffs = xk['rprc_PBr_coeffs'][0]
rqty_MAr_coeffs = xk['rqty_MAr_coeffs'][0]
rprc_MAr_coeffs = xk['rprc_MAr_coeffs'][0]
markup_MAr_coeffs = xk['markup_MAr_coeffs'][0]

In [ ]:
balance = 1000.0
long_psize = 0.0
long_pprice = 0.0
highest_bid = prices[-1]
lowest_ask = prices[-1]
MA_band_lower = prices[-1]
MA_band_upper = prices[-1]
MA_ratios = np.array([1.0, 1.0, 1.0])
available_margin = 2000.0

'''
pbr_limit = config_to_test['long']['pbr_limit']
iqty_const = config_to_test['long']['iqty_const']
iprc_const = config_to_test['long']['iprc_const']
rqty_const = config_to_test['long']['rqty_const']
rprc_const = config_to_test['long']['rprc_const']
rprc_PBr_coeffs = config_to_test['long']['rprc_PBr_coeffs']

'''


orders = []

print("qty          price          psize         pprice        pprice / price ratio    pbr        bkr diff")
for k in range(10):
    long_entry, long_close = calc_long_orders(
        balance,
        long_psize,
        long_pprice,
        highest_bid,
        lowest_ask,
        MA_band_lower,
        MA_band_upper,
        MA_ratios,
        available_margin,

        spot,
        inverse,
        qty_step,
        price_step,
        min_qty,
        min_cost,
        c_mult,
        pbr_stop_loss,
        pbr_limit,
        iqty_const,
        iprc_const,
        rqty_const,
        rprc_const,
        markup_const,
        iqty_MAr_coeffs,
        iprc_MAr_coeffs,
        rprc_PBr_coeffs,
        rqty_MAr_coeffs,
        rprc_MAr_coeffs,
        markup_MAr_coeffs)
    
    
    new_long_psize = long_psize + long_entry[0]
    long_pprice = long_entry[1] * (long_entry[0] / new_long_psize) + long_pprice * (long_psize / new_long_psize)
    long_psize = new_long_psize
    long_pcost = long_psize * long_pprice
    bkr_price = calc_bankruptcy_price(balance,
                          long_psize,
                          long_pprice,
                          0.0,
                          0.0,
                          inverse, c_mult)
    bkr_diff = calc_diff(bkr_price, long_pprice)
    pbr = long_pcost / balance
    if pbr > pbr_limit or long_entry[0] == 0.0:
        break
    print(f"{long_entry[0]: <12} {long_entry[1]: <14} {long_psize: <12}  {round_(long_pprice, price_step): <14}"
          f"{round(long_pprice / long_entry[1], 4): <23} {round(pbr, 4): <10} {round(bkr_diff, 6)}")
    orders.append(long_entry)
print()
orders = numpyize(orders)
entry_prices = orders[:,1].astype(float)
print('grid range', entry_prices[0] / entry_prices[-1])
print('dist between grid entries', entry_prices[:-1] / entry_prices[1:])

In [ ]:
n_days_ = 14
nticks = len(df[df.index > (time() - 60 * 60 * 24 * n_days_) * 1000])
print('n ticks', nticks)
edf = pd.DataFrame({p: np.repeat(p, nticks) for p in entry_prices}, index=df.index[-nticks:]).join(df.price.iloc[-nticks:])
edf.iloc[::100].plot()

In [ ]:
# simplified grid design, ignoring MAr coeffs and rounding
# entry grid is designed by adjusting parameters iqty, rqty, rprc, rqty_pbr_coeffs

In [ ]:
def calc_entry(balance, psize, pprice, initial_entry_price, iqty, rqty, rprc, rqty_pbr_coeffs):
    qty = (balance / initial_entry_price) * iqty
    if psize == 0.0:
        return qty, initial_entry_price
    pbr = psize * pprice / balance
    qty += psize * rqty
    return qty, pprice * (rprc + pbr**2 * rqty_pbr_coeffs[0] + pbr * rqty_pbr_coeffs[1])


In [ ]:
balance = 1000
psize = 0.0
pprice = 0.0
pbr = 0.0
initial_entry_price = 34000.0
iqty = 0.001
rqty = 1.3
rprc = 0.995
pbr_limit = 2.0
rqty_pbr_coeffs = (-0.012, -0.012)
orders = []
while pbr < pbr_limit:
    entry_qty, entry_price = calc_entry(balance, psize, pprice, initial_entry_price, iqty, rqty, rprc, rqty_pbr_coeffs)
    new_psize = psize + entry_qty
    pprice = pprice * (psize / new_psize) + entry_price * (entry_qty / new_psize)
    psize = new_psize
    pbr = psize * pprice / balance
    orders.append((entry_qty, entry_price, psize, pprice, pbr))
    
gdf = pd.DataFrame(orders, columns=['qty', 'price', 'psize', 'pprice', 'pbr'])
gdf.loc[:, 'prev_prc_dist'] = 1 - gdf.price / gdf.price.shift(1)
gdf.loc[:, 'pprice_price_ratio'] = gdf.pprice / gdf.price

print(f'grid range {(1 - gdf.price.iloc[-1] / gdf.price.iloc[0]) * 100:.4}%')
gdf

In [ ]:
# bancor

In [ ]:
from njit_funcs import njit_backtest_bancor

In [ ]:
# test on data whose first price equals last price
if data[0][2] > data[-1][2]:
    for i in range(len(data)):
        if data[i][2] <= data[-1][2]:
            break
else:
    for i in range(len(data)):
        if data[i][2] >= data[-1][2]:
            break
plt.plot(data[i:,2])
i

In [ ]:
# ticks = data
ticks = data[i:]
starting_balance = 2.0 # in terms of quote
latency_simulation_ms = 1000.0 # ms
maker_fee = 0.00075  # with bnb discount
maker_fee = 0.0

qty_step = config['qty_step']
price_step = config['price_step']
min_qty = config['min_qty']
min_cost = config['min_cost']
spans = calc_spans(600, 10000, 3)
spans = calc_spans(1, 1, 3)
qty_pct = 0.000001
bancor_price_spread = 0.01
MA_band_spread = -1.0  # disable ema bands

In [ ]:
fills = njit_backtest_bancor(
    ticks,
    starting_balance,
    latency_simulation_ms,
    maker_fee,
    qty_step,
    price_step,
    min_qty,
    min_cost,
    spans,
    qty_pct,
    bancor_price_spread,
    MA_band_spread
)
fdf = pd.DataFrame(fills)
fdf.columns = ['trade_id', 'timestamp', 'fee_paid', 'fee_token', 'qty', 'price', 'quot_balance', 'coin_balance', 'type']
fdf = fdf.set_index('timestamp')
fdf.loc[:, 'bancor_price'] = fdf.quot_balance / fdf.coin_balance
cgain = fdf.coin_balance.iloc[-1] / fdf.coin_balance.iloc[0]
qgain = fdf.quot_balance.iloc[-1] / fdf.quot_balance.iloc[0]
cqmean = (cgain + qgain) / 2
n_weeks = (fdf.index[-1] - fdf.index[0]) / 1000 / 60 / 60 / 24 / 7
buys = fdf[fdf.type.str.contains('bid')]
sels = fdf[fdf.type.str.contains('ask')]
bvwap = (buys.qty * buys.price).sum() / buys.qty.sum()
svwap = (sels.qty * sels.price).sum() / sels.qty.sum()
vol = fdf.qty.abs().sum()
monthly_vol = vol / (n_weeks / 4)


print(f'cgain   {round_dynamic(cgain, 5)}')
print(f'qgain   {round_dynamic(qgain, 5)}')
print(f'cqmean  {round_dynamic(cqmean, 5)}')
print(f'n weeks {round_dynamic(n_weeks, 4)}')
print()
print(f'weekly  {round_dynamic(cqmean**(1 / n_weeks), 5)}')
print()
print(f'bvwap   {round_dynamic(bvwap, 5)}')
print(f'svwap   {round_dynamic(svwap, 5)}')
print(f'sbratio {round_dynamic(svwap / bvwap, 5)}')
print(f'n_fills {len(fills)}')
print(f'vol     {round_dynamic(vol, 4)}')
print(f'm vol   {round_dynamic(monthly_vol, 4)}')



In [ ]:
dfc = df[df.index >= fdf.index[0]]
dfc.price.iloc[::100].plot()
fdf.bancor_price.plot()
buys.price.plot(style='b.')
sels.price.plot(style='r.')


In [ ]:
quotn = fdf.quot_balance / fdf.quot_balance.iloc[0]
coinn = fdf.coin_balance / fdf.coin_balance.iloc[0]
qcmean = (quotn + coinn) / 2
quotn.plot()
coinn.plot()
qcmean.plot()

In [ ]:
qcmean.plot()

In [ ]:
fdf

In [ ]:
# load external fills to inspect

fdf = pd.read_csv('backtests/binance/BTSUSDT/plots/2021-06-12T114030/fills.csv').set_index('trade_id')

In [ ]:
longs = fdf[fdf.type.str.contains('long')]
shrts = fdf[fdf.type.str.contains('shrt')]

In [ ]:
longs

In [ ]:
longs.tail(60)

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()